In [181]:
import pandas as pd
import openpyxl

file = 'uniprotkb_accession_A1A4S6_OR_accession_2023_08_31.tsv'
structure_columns = ['Helix', 'Turn', 'Beta strand']

def extract_positions(row_data, structure):
    if pd.isna(row_data):
        return []
    items = row_data.split('; ')
    prefix_map = {
        'Helix': 'HELIX',
        'Turn': 'TURN',
        'Beta strand': 'STRAND'
    }
    prefix = prefix_map.get(structure, '')
    return [item.replace(f'{prefix} ', '') for item in items if item.startswith(prefix)]

def extract_subsequences(sequence, positions):
    subsequences = {}
    for pos in positions:
        start, end = map(int, pos.split(".."))
        subseq = sequence[start-1:end]
        subsequences[pos] = subseq
    return subsequences

def autoscale_excel(excel_path):
    book = openpyxl.load_workbook(excel_path)
    sheet = book.active
    
    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                lines = str(cell.value).split('\n')
                longest_line = max(lines, key=len) if lines else ""
                if len(longest_line) > max_length:
                    max_length = len(longest_line)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[openpyxl.utils.get_column_letter(column[0].column)].width = adjusted_width
    
    book.save(excel_path)

In [182]:
data = pd.read_csv(file, sep='\t')
data

,Entry,Helix,Turn,Beta strand,Sequence,Entry Name
0,A1A4S6,"HELIX 780..782; /evidence=""ECO:0007829|PDB:2MIO""",NaN,"STRAND 728..730; /evidence=""ECO:0007829|PDB:2M...",MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...,RHG10_HUMAN
1,A1L3X0,"HELIX 17..20; /evidence=""ECO:0007829|PDB:6Y7F""...","TURN 89..94; /evidence=""ECO:0007829|PDB:6Y7F""","STRAND 97..99; /evidence=""ECO:0007829|PDB:6Y7F""",MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...,ELOV7_HUMAN
2,A2RUC4,"HELIX 16..22; /evidence=""ECO:0007829|PDB:3AL5""...","TURN 70..72; /evidence=""ECO:0007829|PDB:3AL6"";...","STRAND 10..13; /evidence=""ECO:0007829|PDB:3AL5...",MAGQHLPVPRLEGVSREQFMQHLYPQRKPLVLEGIDLGPCTSKWTV...,TYW5_HUMAN
3,A4GXA9,"HELIX 82..86; /evidence=""ECO:0007829|PDB:7F6L""...","TURN 70..72; /evidence=""ECO:0007829|PDB:7F6L"";...","STRAND 73..80; /evidence=""ECO:0007829|PDB:7F6L...",MARVGPGRAGVSCQGRGRGRGGSGQRRPPTWEISDSDAEDSAGSEA...,EME2_HUMAN
4,A6H8Y1,"HELIX 305..317; /evidence=""ECO:0007829|PDB:5N9...",NaN,NaN,MFRRARLSVKPNVRPGVGARGSTASNPQRGRESPRPPDPATDSASK...,BDP1_HUMAN
5,A6NGG8,"HELIX 835..838; /evidence=""ECO:0007829|PDB:7LX...",NaN,NaN,MGCTPSHSDLVNSVAKSGIQFLKKPKAIRPGCQGGSERGSIPLLVK...,PCARE_HUMAN


In [183]:
positions_data = {}
for col in structure_columns:
    positions_data[f'{col} positions'] = data[col].apply(lambda row: extract_positions(row, col))

positions_df = pd.DataFrame(positions_data)
positions_df

,Helix positions,Turn positions,Beta strand positions
0,[780..782],[],"[728..730, 732..737, 743..746, 754..760, 766....."
1,"[17..20, 23..25, 35..49, 51..56, 65..88, 108.....",[89..94],[97..99]
2,"[16..22, 24..26, 40..43, 46..53, 82..90, 121.....","[70..72, 114..116, 205..207]","[10..13, 30..34, 57..66, 76..81, 106..108, 143..."
3,"[82..86, 88..90, 91..97, 147..156, 185..192, 2...","[70..72, 336..340]","[73..80, 102..106, 110..119, 141..145, 178..18..."
4,"[305..317, 322..328, 334..347, 349..357, 364.....",[],[]
5,"[835..838, 842..845]",[],[]


In [184]:
subsequences_data = {}
for col in structure_columns:
    subsequences_data[f'{col} subsequences'] = data.apply(
        lambda row: list(extract_subsequences(row['Sequence'], positions_df.loc[row.name, f'{col} positions']).values()), axis=1
    )

subsequences_df = pd.DataFrame(subsequences_data)
subsequences_df

,Helix subsequences,Turn subsequences,Beta strand subsequences
0,[QNY],[],"[IRS, KARAVY, HSSE, IFEDVQT, WLEGTL, KRGLIP, VKL]"
1,"[WIKD, PRV, LPQTILLGFYVYFVT, LGPKLM, KKAMITYNF...",[SGWGIG],[FRC]
2,"[REQFMQH, YPQ, CTSK, VDYLSQVG, FDQLVQRAA, IRKQ...","[FIS, PRK, LAK]","[RLEG, LVLEG, EVKIHVAAVA, VYRTLP, YLR, FFSSVFR..."
3,"[TAILE, AGA, DVLMEAL, PEEFLQGVAT, LDAYLWSR, WP...","[EQV, GLLAD]","[LKRLAVCV, CECRI, RPARSLRWTR, LLLLL, PHLAVIG, ..."
4,"[NKETDMFFLAISM, FSMIGQL, RIEIKNKFKREEKT, GWRID...",[],[]
5,"[MEVL, SFAS]",[],[]


In [185]:
excel_data = {}
for col in structure_columns:
    excel_data[col] = data.apply(
        lambda row: '\n'.join([f"{pos}-{seq}" for pos, seq in extract_subsequences(row['Sequence'], positions_df.loc[row.name, f'{col} positions']).items()]), axis=1
    )

excel_df = pd.concat([data['Entry'], pd.DataFrame(excel_data)], axis=1)
excel_df

,Entry,Helix,Turn,Beta strand
0,A1A4S6,780..782-QNY,,728..730-IRS\n732..737-KARAVY\n743..746-HSSE\n...
1,A1L3X0,17..20-WIKD\n23..25-PRV\n35..49-LPQTILLGFYVYFV...,89..94-SGWGIG,97..99-FRC
2,A2RUC4,16..22-REQFMQH\n24..26-YPQ\n40..43-CTSK\n46..5...,70..72-FIS\n114..116-PRK\n205..207-LAK,10..13-RLEG\n30..34-LVLEG\n57..66-EVKIHVAAVA\n...
3,A4GXA9,82..86-TAILE\n88..90-AGA\n91..97-DVLMEAL\n147....,70..72-EQV\n336..340-GLLAD,73..80-LKRLAVCV\n102..106-CECRI\n110..119-RPAR...
4,A6H8Y1,305..317-NKETDMFFLAISM\n322..328-FSMIGQL\n334....,,
5,A6NGG8,835..838-MEVL\n842..845-SFAS,,


In [188]:
excel_path = f"./{file.replace('.tsv', '')}.xlsx"
excel_df.to_excel(excel_path, index=False)
autoscale_excel(excel_path)